In [1]:
import os
os.environ['MPLCONFIGDIR'] = "/home/mventura/latent_ode_expts"
import sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import matplotlib.pyplot as plt

import time
import datetime
import argparse
import numpy as np
import pandas as pd
from random import SystemRandom
from sklearn import model_selection

import torch
import torch.nn as nn
from torch.nn.functional import relu
import torch.optim as optim

import lib.utils as utils
from lib.plotting import *

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics

from lib.utils import compute_loss_all_batches
%load_ext autoreload 
%autoreload 2

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-1195ydan because the default path (/home/mventura/latent_ode_expts) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# LatentODE(10)

In [2]:
# !python3 run_models.py --niters 50 -n 1000 -s 1 -l 10 --dataset periodic --latent-ode --noise-weight 0 --viz

In [3]:
# !python3 run_models.py --niters 50 -n 1000 -s 0.5 -l 10 --dataset periodic --latent-ode --noise-weight 0 --viz

latent dim = 10, missing = 0, noise = 0

In [4]:
!python3 run_models.py --niters 50 -n 1000 -s 1 -l 10 --dataset periodic_d2 --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 1 -l 10 --dataset periodic_d2 --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 8700
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 2167.134277 | Likelihood -2217.299072 | KL fp 3.2285 | FP STD 0.9967|
KL coef: 0.0
Train loss (one batch): 2384.998291015625
Train CE loss (one batch): 0.0
Test MSE: 0.4442
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 8700
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 1821.191528 | Likelihood -1824.483521 | KL fp 5.3345 | FP STD 0.8278|
KL coef: 0.0
Train loss (one batch): 1707.29248046875
Train CE loss (one batch): 0.0
Test MSE: 0.3656
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 8700
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 1781.687744 | Likelihood -1785.542358 | KL fp 5.2039 | FP STD 0.6886|
KL coef: 0.0
Train loss (one bat

plotting....
Computing loss... 0
Experiment 8700
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 1323.322510 | Likelihood -1323.528809 | KL fp 6.7504 | FP STD 0.0819|
KL coef: 0.15705680661607324
Train loss (one batch): 1277.0877685546875
Train CE loss (one batch): 0.0
Test MSE: 0.2654
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 8700
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 1313.210815 | Likelihood -1313.398560 | KL fp 6.8041 | FP STD 0.0735|
KL coef: 0.1654862385499125
Train loss (one batch): 1265.429443359375
Train CE loss (one batch): 0.0
Test MSE: 0.2634
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 8700
Epoch 0029 [Test seq (cond on sampled tp)] | Loss 1304.765259 | Likelihood -1304.874756 | KL fp 6.9097 | FP STD 0.0674|
KL coef: 0.17383137616441335
Train loss (one batch): 1253.845947265625
Train CE loss (one batch): 0.0
Test MSE: 0.2617
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing lo

latent dim = 10, missing = 50, noise = 0

In [5]:
!python3 run_models.py --niters 50 -n 1000 -s 0.5 -l 10 --dataset periodic_d2 --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 0.5 -l 10 --dataset periodic_d2 --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 49974
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 1937.670532 | Likelihood -1945.553223 | KL fp 4.6390 | FP STD 1.2551|
KL coef: 0.0
Train loss (one batch): 1860.6644287109375
Train CE loss (one batch): 0.0
Test MSE: 0.3898
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 49974
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 1796.585083 | Likelihood -1798.606079 | KL fp 4.8714 | FP STD 0.6223|
KL coef: 0.0
Train loss (one batch): 1714.5863037109375
Train CE loss (one batch): 0.0
Test MSE: 0.3605
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 49974
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 1759.510376 | Likelihood -1765.725464 | KL fp 4.4886 | FP STD 0.7047|
KL coef: 0.0
Train loss 

Experiment 49974
Epoch 0026 [Test seq (cond on sampled tp)] | Loss 1335.771118 | Likelihood -1336.548218 | KL fp 6.5063 | FP STD 0.0983|
KL coef: 0.14854222890512447
Train loss (one batch): 1300.8480224609375
Train CE loss (one batch): 0.0
Test MSE: 0.2680
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 49974
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 1320.225708 | Likelihood -1320.786621 | KL fp 6.4050 | FP STD 0.0947|
KL coef: 0.15705680661607324
Train loss (one batch): 1275.5059814453125
Train CE loss (one batch): 0.0
Test MSE: 0.2649
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 49974
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 1296.315186 | Likelihood -1296.847168 | KL fp 6.3944 | FP STD 0.0952|
KL coef: 0.1654862385499125
Train loss (one batch): 1250.5052490234375
Train CE loss (one batch): 0.0
Test MSE: 0.2601
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 49974
Epo

# LatentODE(5)

In [6]:
# !python3 run_models.py --niters 50 -n 1000 -s 1 -l 5 --dataset periodic --latent-ode --noise-weight 0 --viz

In [7]:
# !python3 run_models.py --niters 50 -n 1000 -s 0.5 -l 5 --dataset periodic --latent-ode --noise-weight 0 --viz

latent dim = 5, missing = 0, noise = 0

In [8]:
!python3 run_models.py --niters 50 -n 1000 -s 1 -l 5 --dataset periodic_d2 --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 1 -l 5 --dataset periodic_d2 --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 5455
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 1949.360352 | Likelihood -1952.240356 | KL fp 4.7589 | FP STD 0.3678|
KL coef: 0.0
Train loss (one batch): 1926.5887451171875
Train CE loss (one batch): 0.0
Test MSE: 0.3912
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 5455
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 1788.415527 | Likelihood -1790.585449 | KL fp 5.7416 | FP STD 0.4210|
KL coef: 0.0
Train loss (one batch): 1708.9453125
Train CE loss (one batch): 0.0
Test MSE: 0.3589
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 5455
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 1767.953003 | Likelihood -1770.230835 | KL fp 5.6193 | FP STD 0.4342|
KL coef: 0.0
Train loss (one batch):

plotting....
Computing loss... 0
Experiment 5455
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 1466.093750 | Likelihood -1466.254883 | KL fp 8.7896 | FP STD 0.0752|
KL coef: 0.15705680661607324
Train loss (one batch): 1418.639404296875
Train CE loss (one batch): 0.0
Test MSE: 0.2940
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 5455
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 1452.269409 | Likelihood -1451.899902 | KL fp 9.0981 | FP STD 0.0631|
KL coef: 0.1654862385499125
Train loss (one batch): 1402.7813720703125
Train CE loss (one batch): 0.0
Test MSE: 0.2911
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 5455
Epoch 0029 [Test seq (cond on sampled tp)] | Loss 1431.903931 | Likelihood -1432.311157 | KL fp 8.8273 | FP STD 0.0843|
KL coef: 0.17383137616441335
Train loss (one batch): 1383.38330078125
Train CE loss (one batch): 0.0
Test MSE: 0.2872
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing los

latent dim = 5, missing = 50, noise = 0

In [9]:
!python3 run_models.py --niters 50 -n 1000 -s 0.5 -l 5 --dataset periodic_d2 --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 0.5 -l 5 --dataset periodic_d2 --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 74547
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 1835.951172 | Likelihood -1844.896484 | KL fp 5.8281 | FP STD 0.6203|
KL coef: 0.0
Train loss (one batch): 1836.8017578125
Train CE loss (one batch): 0.0
Test MSE: 0.3697
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 74547
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 1779.160767 | Likelihood -1780.899780 | KL fp 5.7258 | FP STD 0.3812|
KL coef: 0.0
Train loss (one batch): 1721.2059326171875
Train CE loss (one batch): 0.0
Test MSE: 0.3569
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 74547
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 1741.222290 | Likelihood -1742.375488 | KL fp 6.0167 | FP STD 0.3777|
KL coef: 0.0
Train loss (one

Experiment 74547
Epoch 0026 [Test seq (cond on sampled tp)] | Loss 1349.943604 | Likelihood -1349.652954 | KL fp 9.7202 | FP STD 0.0569|
KL coef: 0.14854222890512447
Train loss (one batch): 1309.76123046875
Train CE loss (one batch): 0.0
Test MSE: 0.2707
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 74547
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 1343.450195 | Likelihood -1343.039185 | KL fp 9.6247 | FP STD 0.0619|
KL coef: 0.15705680661607324
Train loss (one batch): 1300.266845703125
Train CE loss (one batch): 0.0
Test MSE: 0.2693
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 74547
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 1333.434937 | Likelihood -1332.867432 | KL fp 10.1686 | FP STD 0.0380|
KL coef: 0.1654862385499125
Train loss (one batch): 1287.399658203125
Train CE loss (one batch): 0.0
Test MSE: 0.2673
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 74547
Epoch 